# 入门
### Hello World

In [ ]:
import tensorflow as tf
c = tf.constant('hello')
with tf.Session() as sess:
    print(sess.run(c))

### add与+的区别？
实际上最终调用是一个函数，都到了gen_math_ops.add。
tf.add直接调用库方法，+是重载了__add__符号，在tensorflow/python/ops/math_ops.py中定义了_OverrideBinaryOperatorHelper(gen_math_ops.add, "add")，也就是说最终还是调用了tf.add

加载模块tensorflow/python/ops/math_ops.py
```
tensorflow/python/ops/math_ops.py： _OverrideBinaryOperatorHelper(gen_math_ops.add, "add") 
-> tensorflow/python/ops/math_ops.py： clazz_object._override_operator("__%s__" % op_name, binary_op_wrapper) 在binary_op_wrapper中会实际调用gen_math_ops.add
-> tensorflow/python/framework/ops.py： _override_helper(Tensor, operator, func)
-> tensorflow/python/framework/ops.py： setattr(clazz_object, operator, func)
```

* Python 中的setattr有点像Ruby元编程中的动态方法，既可以是属性值，也可以是函数方法。setattr(obj, '__add__', add)也就是动态重载+运算符*

所有重载的运算符在Tensor.OVERLOADABLE_OPERATORS:中进行了定义。

参考：
https://stackoverflow.com/questions/37900780/in-tensorflow-what-is-the-difference-between-tf-add-and-operator

In [20]:
import tensorflow as tf
a = tf.constant(1)
b = tf.constant(2)
with tf.Session() as sess:
    print(sess.run(a+b))
with tf.Session() as sess:
    print(sess.run(tf.add(a,b)))

3
3


### 一个简单的线性模型

In [4]:
import tensorflow as tf
import numpy as np

# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

# 构造一个线性模型
# 
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

# 最小化方差
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()

# 启动图 (graph)
sess = tf.Session()
sess.run(init)

# 拟合平面
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

0 [[-0.10845557  0.80493349]] [ 0.21462715]
20 [[ 0.06473533  0.35856053]] [ 0.23327331]
40 [[ 0.0984279   0.24692839]] [ 0.27515358]
60 [[ 0.10159053  0.2144341 ]] [ 0.29116544]
80 [[ 0.10097016  0.20456707]] [ 0.29693624]
100 [[ 0.10041523  0.20147374]] [ 0.29895264]
120 [[ 0.10015769  0.20048185]] [ 0.29964495]
140 [[ 0.10005664  0.20015888]] [ 0.29988027]
160 [[ 0.10001975  0.20005268]] [ 0.29995975]
180 [[ 0.10000679  0.20001753]] [ 0.29998648]
200 [[ 0.1000023   0.20000584]] [ 0.29999545]
